In [39]:
import pandas as pd
import numpy as np
import os
# plt.style.use('seaborn-colorblind')
# %matplotlib inline
from feature_selection import filter_method as ft

In [40]:
df = pd.read_csv('./data.csv')
df.head(5)

,uid,all_bal,avg_mth_x,avg_qur_x,avg_year_x,sa_bal,td_bal,fin_bal,sa_crd_bal,td_crd_bal,...,deps_type,acct_sts,frz_sts,stp_sts,acct_bal,bal,avg_mth_y,avg_qur_y,avg_year_y,star_level
0,330801199010011879,4.42,0.53,0.0,4.42,4.42,0.0,0.0,4.42,0.0,...,0,0,0,0,4.42,4.42,0.53,0.0,4.42,1
1,320982199302235614,20.19,2.41,0.0,20.19,20.19,0.0,0.0,20.19,0.0,...,0,0,0,0,20.19,20.19,2.41,0.0,20.19,1
2,320982199003129328,60281.80,7197.83,0.0,57871.58,281.80,60000.0,0.0,281.80,0.0,...,0,0,0,0,281.80,281.80,33.65,0.0,267.92,3
3,320925196507290894,4364.46,521.13,0.0,8505.26,4364.46,0.0,0.0,4364.46,0.0,...,0,0,0,0,4364.46,4364.46,521.13,0.0,8505.26,2
4,320721198511044525,29.72,3.55,0.0,214.09,29.72,0.0,0.0,29.72,0.0,...,0,0,0,0,29.72,29.72,3.55,0.0,214.09,1


In [41]:
x = df.iloc[:, 0:-1]
x.head(5)

,uid,all_bal,avg_mth_x,avg_qur_x,avg_year_x,sa_bal,td_bal,fin_bal,sa_crd_bal,td_crd_bal,...,acct_char,deps_type,acct_sts,frz_sts,stp_sts,acct_bal,bal,avg_mth_y,avg_qur_y,avg_year_y
0,330801199010011879,4.42,0.53,0.0,4.42,4.42,0.0,0.0,4.42,0.0,...,0,0,0,0,0,4.42,4.42,0.53,0.0,4.42
1,320982199302235614,20.19,2.41,0.0,20.19,20.19,0.0,0.0,20.19,0.0,...,0,0,0,0,0,20.19,20.19,2.41,0.0,20.19
2,320982199003129328,60281.80,7197.83,0.0,57871.58,281.80,60000.0,0.0,281.80,0.0,...,0,0,0,0,0,281.80,281.80,33.65,0.0,267.92
3,320925196507290894,4364.46,521.13,0.0,8505.26,4364.46,0.0,0.0,4364.46,0.0,...,0,0,0,0,0,4364.46,4364.46,521.13,0.0,8505.26
4,320721198511044525,29.72,3.55,0.0,214.09,29.72,0.0,0.0,29.72,0.0,...,0,0,0,0,0,29.72,29.72,3.55,0.0,214.09


In [42]:
y = df.iloc[:, -1]
y.head(5)

0    1
1    1
2    3
3    2
4    1
Name: star_level, dtype: int64

In [43]:
# filter_method: 剔除常量，因为这些特征对于 y 值没有任何的影响
constant_feature = ft.constant_feature_detect(x, threshold=0.9)
constant_feature

13  variables are found to be almost constant


['avg_qur_x',
 'fin_bal',
 'td_crd_bal',
 'sa_td_bal',
 'ntc_bal',
 'td_3m_bal',
 'td_6m_bal',
 'td_2y_bal',
 'out_td_bal',
 'cd_bal',
 'frz_sts',
 'stp_sts',
 'avg_qur_y']

In [44]:
x.drop(labels=constant_feature, axis=1, inplace=True)
# uid 非数字，对 star_level 也没有影响，去除
x.drop(labels='uid', axis=1, inplace=True)
x.head(5)

,all_bal,avg_mth_x,avg_year_x,sa_bal,td_bal,sa_crd_bal,td_1y_bal,td_3y_bal,acct_char,deps_type,acct_sts,acct_bal,bal,avg_mth_y,avg_year_y
0,4.42,0.53,4.42,4.42,0.0,4.42,0.0,0.0,0,0,0,4.42,4.42,0.53,4.42
1,20.19,2.41,20.19,20.19,0.0,20.19,0.0,0.0,0,0,0,20.19,20.19,2.41,20.19
2,60281.80,7197.83,57871.58,281.80,60000.0,281.80,0.0,60000.0,0,0,0,281.80,281.80,33.65,267.92
3,4364.46,521.13,8505.26,4364.46,0.0,4364.46,0.0,0.0,0,0,0,4364.46,4364.46,521.13,8505.26
4,29.72,3.55,214.09,29.72,0.0,29.72,0.0,0.0,0,0,0,29.72,29.72,3.55,214.09


In [45]:
# 计算相关系数，相关系数大于 0.7 的特征可以删除
corr = ft.corr_feature_detect(data=x, threshold=0.7)
for i in corr:
    print(i, '\n')

      feature1   feature2      corr
0   avg_year_x  avg_mth_x  0.986980
15  avg_year_x    all_bal  0.946326
16  avg_year_x     td_bal  0.931101
31  avg_year_x  td_3y_bal  0.700067 

     feature1 feature2     corr
2  sa_crd_bal   sa_bal  0.98572 

     feature1    feature2      corr
6   avg_mth_y         bal  0.973440
8   avg_mth_y    acct_bal  0.973440
19  avg_mth_y  avg_year_y  0.851603 



In [46]:
corr_columns = ['avg_mth_x', 'sa_crd_bal', 'avg_mth_y']
x.drop(columns=corr_columns, axis=1, inplace=True)

In [47]:
# 计算 VIF 共线性，共线性大于 20 的特征删除
vif = ft.calc_VIF(x)
vif.head(30)

/opt/homebrew/Caskroom/miniconda/base/envs/ml-pytorch/lib/python3.9/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,index,VIF
0,all_bal,79.609039
1,avg_year_x,16.624898
2,sa_bal,4.490780
3,td_bal,68.585476
4,td_1y_bal,1.755131
5,td_3y_bal,6.575741
6,acct_char,1.548635
7,deps_type,2.165397
8,acct_sts,1.538249
9,acct_bal,inf


In [48]:
# all_bal、td_bal、acct_bal、bal 共线性太强，剔除
vif_columns = ['all_bal', 'td_bal', 'acct_bal', 'bal']
x.drop(columns=vif_columns, axis=1, inplace=True)

In [49]:
# 互信息衡量选择最好的 80% 特征
mi = ft.mutual_info(X=x, y=y, select_k=0.8)
print(mi)

Index(['avg_year_x', 'sa_bal', 'td_1y_bal', 'td_3y_bal', 'acct_sts',
       'avg_year_y'],
      dtype='object')


In [50]:
new_x = x[['avg_year_x', 'sa_bal', 'td_1y_bal', 'td_3y_bal', 'deps_type', 'avg_year_y']]
new_x.head(5)

,avg_year_x,sa_bal,td_1y_bal,td_3y_bal,deps_type,avg_year_y
0,4.42,4.42,0.0,0.0,0,4.42
1,20.19,20.19,0.0,0.0,0,20.19
2,57871.58,281.80,0.0,60000.0,0,267.92
3,8505.26,4364.46,0.0,0.0,0,8505.26
4,214.09,29.72,0.0,0.0,0,214.09


In [51]:
ans = pd.concat([new_x, y], axis=1)
ans.to_csv('new_data.csv', header=True, index=False, sep=',')